# Question 1
## Use Random Forest to prepare a model on fraud data 
## treating those who have taxable_income <= 30000 as "Risky" and others are "Good"


In [19]:
# Random Forest Classification
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn import preprocessing

In [20]:
fraud =pd.read_csv('Fraud_check.csv')
fraud


,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [24]:
label_encoder = preprocessing.LabelEncoder()
fraud['Urban']= label_encoder.fit_transform(fraud['Urban']) 

In [25]:
fraud.rename(columns={"Marital.Status":"Marital_Status","Taxable.Income":"Taxable_Income","City.Population":"City_Population","Work.Experience":"Work_Experience"}, inplace= True)

In [26]:
fraud.head()

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban
0,NO,Single,68833,50047,10,1
1,YES,Divorced,33700,134075,18,1
2,NO,Married,36925,160205,30,1
3,YES,Single,50190,193264,15,1
4,NO,Married,81002,27533,28,0


In [29]:
label_encoder = preprocessing.LabelEncoder()
fraud['Undergrad']= label_encoder.fit_transform(fraud['Undergrad'])
fraud['Marital_Status']= label_encoder.fit_transform(fraud['Marital_Status'])

In [30]:
fraud.head()

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban
0,0,2,68833,50047,10,1
1,1,0,33700,134075,18,1
2,0,1,36925,160205,30,1
3,1,2,50190,193264,15,1
4,0,1,81002,27533,28,0


In [32]:
fraud["TaxInc"] = pd.cut(fraud["Taxable_Income"], bins = [10002,30000,99620], labels = ["Risky", "Good"])

In [33]:
fraud = pd.get_dummies(fraud,columns = ["TaxInc"],drop_first=True)

In [34]:
fraud

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban,TaxInc_Good
0,0,2,68833,50047,10,1,1
1,1,0,33700,134075,18,1,1
2,0,1,36925,160205,30,1,1
3,1,2,50190,193264,15,1,1
4,0,1,81002,27533,28,0,1
...,...,...,...,...,...,...,...
595,1,0,76340,39492,7,1,1
596,1,0,69967,55369,2,1,1
597,0,0,47334,154058,0,1,1
598,1,1,98592,180083,17,0,1


In [35]:
def norm_func(i):
    x = (i-i.min())/(i.max()-i.min())
    return (x)

In [37]:
fraud_norm = norm_func(fraud[['Undergrad','Marital_Status','City_Population','Work_Experience','Urban','TaxInc_Good']])
fraud_norm.tail(10)

,Undergrad,Marital_Status,City_Population,Work_Experience,Urban,TaxInc_Good
590,0.0,0.5,0.341473,0.466667,1.0,1.0
591,1.0,1.0,0.615406,0.600000,1.0,0.0
592,1.0,1.0,0.283703,0.533333,0.0,1.0
593,0.0,0.0,0.610256,0.333333,1.0,1.0
594,0.0,0.5,0.412341,0.300000,1.0,1.0
595,1.0,0.0,0.078811,0.233333,1.0,1.0
596,1.0,0.0,0.170058,0.066667,1.0,1.0
597,0.0,0.0,0.737240,0.000000,1.0,1.0
598,1.0,0.5,0.886810,0.566667,0.0,1.0
599,0.0,0.0,0.760683,0.533333,0.0,1.0


In [56]:
x=fraud_norm.iloc[:,0:5]
y=fraud_norm['TaxInc_Good']
num_trees = 100
max_features = 3
kfold = KFold(n_splits=10, random_state=7,shuffle=True)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = cross_val_score(model, x, y, cv=kfold)
print(results.mean())

0.735


In [57]:
x

,Undergrad,Marital_Status,City_Population,Work_Experience,Urban
0,0.0,1.0,0.139472,0.333333,1.0
1,1.0,0.0,0.622394,0.600000,1.0
2,0.0,0.5,0.772568,1.000000,1.0
3,1.0,1.0,0.962563,0.500000,1.0
4,0.0,0.5,0.010081,0.933333,0.0
...,...,...,...,...,...
595,1.0,0.0,0.078811,0.233333,1.0
596,1.0,0.0,0.170058,0.066667,1.0
597,0.0,0.0,0.737240,0.000000,1.0
598,1.0,0.5,0.886810,0.566667,0.0


In [58]:
y

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
595    1.0
596    1.0
597    1.0
598    1.0
599    1.0
Name: TaxInc_Good, Length: 600, dtype: float64

In [59]:
results

array([0.76666667, 0.68333333, 0.81666667, 0.81666667, 0.71666667,
       0.7       , 0.7       , 0.61666667, 0.78333333, 0.75      ])

# Question 2
## Random Forest
## A cloth manufacturing company is interested to know about the segment or attributes causes high sale. 
## Approach - A Random Forest can be built with target variable Sales (we will first convert it in categorical variable) & all other variable will be independent in the analysis.  

In [61]:
Sales=pd.read_csv('Company_Data.csv')

In [62]:
Sales.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [63]:
Sales['ShelveLoc']= label_encoder.fit_transform(Sales['ShelveLoc'])
Sales['Urban']= label_encoder.fit_transform(Sales['Urban'])
Sales['US']= label_encoder.fit_transform(Sales['US'])

In [64]:
Sales.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,0,42,17,1,1
1,11.22,111,48,16,260,83,1,65,10,1,1
2,10.06,113,35,10,269,80,2,59,12,1,1
3,7.40,117,100,4,466,97,2,55,14,1,1
4,4.15,141,64,3,340,128,0,38,13,1,0


In [65]:
Sales['Sales'].max()

16.27

In [66]:
Sales["Sales_Report"] = pd.cut(Sales["Sales"], bins = [0.0,8.5,16.27], labels = ["High", "Low"])

In [67]:
Sales= pd.get_dummies(Sales,columns = ["Sales_Report"],drop_first=True)

In [68]:
def norm_func(i):
    x = (i-i.min())/(i.max()-i.min())
    return (x)

In [69]:
Sales_norm = norm_func(Sales.iloc[:,:]) 
Sales_norm.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,Sales_Report_Low
0,0.583897,0.622449,0.525253,0.379310,0.533066,0.574850,0.0,0.309091,0.875,1.0,1.0,1.0
1,0.689613,0.346939,0.272727,0.551724,0.501002,0.353293,0.5,0.727273,0.000,1.0,1.0,1.0
2,0.618316,0.367347,0.141414,0.344828,0.519038,0.335329,1.0,0.618182,0.250,1.0,1.0,1.0
3,0.454825,0.408163,0.797980,0.137931,0.913828,0.437126,1.0,0.545455,0.500,1.0,1.0,0.0
4,0.255071,0.653061,0.434343,0.103448,0.661323,0.622754,0.0,0.236364,0.375,1.0,0.0,0.0


In [73]:
x1=Sales_norm.iloc[:,1:11]
y1=Sales_norm.iloc[:,11]
num_trees = 100
max_features = 3
kfold = KFold(n_splits=10, random_state=7,shuffle=True)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = cross_val_score(model, x, y, cv=kfold)
print(results.mean())

0.7383333333333333


In [74]:
x1

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,0.622449,0.525253,0.379310,0.533066,0.574850,0.0,0.309091,0.875,1.0,1.0
1,0.346939,0.272727,0.551724,0.501002,0.353293,0.5,0.727273,0.000,1.0,1.0
2,0.367347,0.141414,0.344828,0.519038,0.335329,1.0,0.618182,0.250,1.0,1.0
3,0.408163,0.797980,0.137931,0.913828,0.437126,1.0,0.545455,0.500,1.0,1.0
4,0.653061,0.434343,0.103448,0.661323,0.622754,0.0,0.236364,0.375,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
395,0.622449,0.878788,0.586207,0.386774,0.622754,0.5,0.145455,0.500,1.0,1.0
396,0.632653,0.020202,0.103448,0.054108,0.574850,1.0,0.545455,0.125,0.0,1.0
397,0.867347,0.050505,0.413793,0.717435,0.808383,1.0,0.272727,1.000,1.0,1.0
398,0.234694,0.585859,0.241379,0.549098,0.425150,0.0,0.454545,0.250,1.0,1.0


In [75]:
y1

0      1.0
1      1.0
2      1.0
3      0.0
4      0.0
      ... 
395    1.0
396    0.0
397    0.0
398    0.0
399    1.0
Name: Sales_Report_Low, Length: 400, dtype: float64

In [76]:
results

array([0.73333333, 0.66666667, 0.83333333, 0.81666667, 0.73333333,
       0.7       , 0.71666667, 0.63333333, 0.78333333, 0.76666667])